In [ ]:
# Instalando as bibliotecas para o web scraping

%pip install selenium
%pip install beautifulsoup4

In [ ]:
# Importando as Bibliotecas

from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from multiprocessing.dummy import Pool as ThreadPool
import time

In [ ]:
def pegar_conteudo(url):
    dados_curso = [url.split('/')[4]]

    driver = webdriver.Firefox(options=opcoes)
    driver.get(url)
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'CourseAbout__header__title')))
    except:
        driver.quit()
        return None
    conteudo = BeautifulSoup(driver.page_source, 'html.parser')    
    driver.quit()
    try:
        nome_curso = conteudo.find('h1', class_='CourseAbout__header__title').text
    except:
        nome_curso = ''

    if len(nome_curso) == 0:
        nome_curso = 'Nome não disponível'
    dados_curso.append(nome_curso)

    try:
        num_alunos = int(conteudo.find('div', class_='CourseAbout__header__info CourseAbout__header__info--students').text[:-7])
    except:
        num_alunos = 'Número de alunos não disponível'
    dados_curso.append(num_alunos)

    return dados_curso

def pegar_dados(curso_max):

    base_url = 'https://app.sapium.com.br/course/{}/about'
    urls = [base_url.format(i) for i in range(1, curso_max)]

    global opcoes
    opcoes = webdriver.FirefoxOptions()

    pool = ThreadPool(6)
    dados_cursos = pool.map(pegar_conteudo, urls)
    pool.close()
    pool.join()
    dados_cursos = [dado for dado in dados_cursos if dado]
    cursos_df = pd.DataFrame(dados_cursos, columns=['Número', 'Nome', 'Número de Alunos'])
    cursos_df = cursos_df.set_index('Número')

    return cursos_df

cursos_df = pegar_dados(10)
cursos_df

In [ ]:
cursos_df[cursos_df["Número de Alunos"] > 1000]